## Deliverable 2. Create a Customer Travel Destinations Map.

In [11]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gmaps_key

# Configure gmaps API key
gmaps.configure(api_key=gmaps_key)

In [12]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Lat Yao,15.7510,99.7893,83.71,75,78,2.44,TH,2021-10-07 02:44:00,broken clouds
1,1,Broome,42.2506,-75.8330,62.56,80,1,4.61,US,2021-10-07 02:44:00,clear sky
2,2,Kirkuk,35.4681,44.3922,70.99,24,0,6.24,IQ,2021-10-07 02:44:00,clear sky
3,3,Kyra,49.5776,111.9753,46.81,53,94,2.93,RU,2021-10-07 02:44:01,overcast clouds
4,4,Barrow,71.2906,-156.7887,33.82,86,90,6.91,US,2021-10-07 02:44:01,mist


In [13]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
preferred_cities_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
3,3,Kyra,49.5776,111.9753,46.81,53,94,2.93,RU,2021-10-07 02:44:01,overcast clouds
28,28,Thompson,55.7435,-97.8558,46.56,71,75,6.91,CA,2021-10-07 02:44:08,broken clouds
41,41,Mar Del Plata,-38.0023,-57.5575,48.99,70,75,11.01,AR,2021-10-07 02:44:11,broken clouds
70,70,Rocha,-34.4833,-54.3333,48.43,62,43,12.26,UY,2021-10-07 02:44:19,scattered clouds
94,94,Chapais,49.7834,-74.8492,49.78,86,85,4.41,CA,2021-10-07 02:44:26,overcast clouds


In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Date                   0
Current Description    0
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.isnull
clean_df = preferred_cities_df

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kyra,RU,46.81,overcast clouds,49.5776,111.9753,
28,Thompson,CA,46.56,broken clouds,55.7435,-97.8558,
41,Mar Del Plata,AR,48.99,broken clouds,-38.0023,-57.5575,
70,Rocha,UY,48.43,scattered clouds,-34.4833,-54.3333,
94,Chapais,CA,49.78,overcast clouds,49.7834,-74.8492,
122,Krasnyy Chikoy,RU,46.53,overcast clouds,50.3643,108.7512,
146,Chuy,UY,48.09,few clouds,-33.6971,-53.4616,
148,Qaqortoq,GL,45.10,scattered clouds,60.7167,-46.0333,
196,Kodiak,US,46.45,scattered clouds,57.7900,-152.4072,
213,Rognan,NO,48.34,light rain,67.0953,15.3878,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gmaps_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json' 

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        continue
        

In [19]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [20]:
# 8a. Create the output File (CSV)
clean_hotel_df = hotel_df

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv("clean_hotels.csv", index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))